# **Chapter 19. Training and Deploying TensorFlow Models at Scale**

## Menyajikan Model TensorFlow 
Setelah model TensorFlow dilatih, model tersebut perlu dimasukkan ke dalam produksi. Ini bisa sesederhana menjalankan model pada batch data, tetapi seringkali lebih terlibat, seperti membungkus model dalam layanan web.

# Setup
Pertama-tama, kita perlu mengimpor beberapa modul yang umum digunakan, memastikan Matplotlib menampilkan gambar secara inline, dan menyiapkan fungsi untuk menyimpan gambar. Kita juga memeriksa apakah Python 3.5 atau lebih baru terpasang (meskipun Python 2.x mungkin berfungsi, itu sudah tidak didukung lagi, jadi kami sangat menyarankan Anda menggunakan Python 3), serta Scikit-Learn ≥0.20 dan TensorFlow ≥2.0.


In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Apakah notebook ini berjalan di Colab atau Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if IS_COLAB or IS_KAGGLE:
    !echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" > /etc/apt/sources.list.d/tensorflow-serving.list
    !curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
    !apt update && apt-get install -y tensorflow-model-server
    %pip install -q -U tensorflow-serving-api

# Diperlukan scikit-learn ≥0.20
import sklearn
assert sklearn.__version__ >= "0.20"

# Diperlukan tensorflow ≥2.0
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Impor umum
import numpy as np
import os

# untuk membuat output notebook ini stabil di seluruh berjalan
np.random.seed(42)
tf.random.set_seed(42)

# Untuk merencanakan angka cantik
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Di mana menyimpan angka
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deploy"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Menyebarkan model TensorFlow ke TensorFlow Serving (TFS)

TF Serving adalah server model berperforma tinggi yang ditulis dalam C++. Ia dapat melayani berbagai versi model Anda, secara otomatis menerapkan versi terbaru, dan lainnya.
* **Mengekspor SavedModels**:
    * Model TensorFlow diekspor ke format **SavedModel** menggunakan `tf.saved_model.save(model, model_path)` atau `model.save(model_path)` (jika ekstensi file bukan `.h5`).
    * SavedModel menyimpan grafik komputasi model dan bobotnya. Sebaiknya sertakan semua lapisan pra-pemrosesan dalam model akhir yang diekspor.
    * SavedModel direpresentasikan sebagai direktori yang berisi file `saved_model.pb` (grafik komputasi) dan subdirektori `variables` (nilai variabel). Mungkin juga menyertakan subdirektori `assets` untuk data tambahan.
    * SavedModel dapat dimuat menggunakan `tf.saved_model.load()` (mengembalikan objek SavedModel) atau `keras.models.load_model()` (mengembalikan model Keras).
    * Alat baris perintah `saved_model_cli` dapat digunakan untuk memeriksa SavedModels dan membuat prediksi. SavedModel berisi satu atau lebih *metagraph*, yang merupakan grafik komputasi ditambah beberapa definisi tanda tangan fungsi.
* **Menginstal TensorFlow Serving**: Opsi yang disarankan adalah menggunakan Docker.
    ```bash
    docker pull tensorflow/serving
    docker run -it --rm -p 8500:8500 -p 8501:8501 \
      -v "/path/to/my_mnist_model:/models/my_mnist_model" \
      -e MODEL_NAME=my_mnist_model \
      tensorflow/serving
    ```
    Perintah ini menjalankan TF Serving, memuat model MNIST, dan menyajikannya melalui gRPC (port 8500) dan REST (port 8501).
* **Mengirim Kueri ke TF Serving melalui REST API**:
    * Kueri harus dalam format JSON dan berisi nama tanda tangan fungsi serta data inpu.
    * Permintaan HTTP POST dikirim ke server. REST API sederhana tetapi bisa tidak efisien untuk data besar karena berbasis teks (JSON).
* **Mengirim Kueri ke TF Serving melalui gRPC API**:
    * Lebih efisien untuk data besar karena menggunakan format biner ringkas dan protokol komunikasi yang efisien (berbasis HTTP/2).
    * Memerlukan *protobuf* `PredictRequest` sebagai input dan menghasilkan *protobuf* `PredictResponse`.
    * Membutuhkan pustaka `tensorflow-serving-api` dan `grpcio`.
* **Menerapkan Versi Model Baru**: TF Serving secara otomatis mendeteksi versi model baru di repositori model dan menangani transisi dengan baik. Ini dapat mengosongkan versi sebelumnya sebelum memuat yang baru atau memuat yang baru sambil tetap melayani permintaan yang tertunda dengan yang lama. TF Serving juga memiliki kemampuan *batching* otomatis (--enable_batching).

Kita akan menggunakan API REST atau API GRPC.

## Simpan/muat `saveDmodel`

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

In [3]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 2s 1ms/step - loss: 1.1140 - accuracy: 0.7066 - val_loss: 0.3715 - val_accuracy: 0.9024
Epoch 2/10
1719/1719 [==============================] - 1s 713us/step - loss: 0.3695 - accuracy: 0.8981 - val_loss: 0.2990 - val_accuracy: 0.9144
Epoch 3/10
1719/1719 [==============================] - 1s 718us/step - loss: 0.3154 - accuracy: 0.9100 - val_loss: 0.2651 - val_accuracy: 0.9272
Epoch 4/10
1719/1719 [==============================] - 1s 706us/step - loss: 0.2765 - accuracy: 0.9223 - val_loss: 0.2436 - val_accuracy: 0.9334
Epoch 5/10
1719/1719 [==============================] - 1s 711us/step - loss: 0.2556 - accuracy: 0.9276 - val_loss: 0.2257 - val_accuracy: 0.9364
Epoch 6/10
1719/1719 [==============================] - 1s 715us/step - loss: 0.2367 - accuracy: 0.9321 - val_loss: 0.2121 - val_accuracy: 0.9396
Epoch 7/10
1719/1719 [==============================] - 1s 729us/step - loss: 0.2198 - accuracy: 0.9390 - val_loss: 0.1970 - v

In [4]:
np.round(model.predict(X_new), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [5]:
model_version = "0001"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
model_path

'my_mnist_model/0001'

In [6]:
import shutil

shutil.rmtree(model_name)

In [7]:
tf.saved_model.save(model, model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


In [8]:
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

my_mnist_model/
    0001/
        saved_model.pb
        variables/
            variables.data-00000-of-00001
            variables.index
        assets/


In [9]:
!saved_model_cli show --dir {model_path}

The given SavedModel contains the following tag-sets:
'serve'


In [10]:
!saved_model_cli show --dir {model_path} --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [11]:
!saved_model_cli show --dir {model_path} --tag_set serve \
                      --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28, 1)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [12]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          flatte

Mari kita tulis contoh baru ke file `npy` sehingga kita dapat meneruskannya dengan mudah ke model kita:

In [13]:
np.save("my_mnist_tests.npy", X_new)

In [14]:
input_name = model.input_names[0]
input_name

'flatten_input'

Dan sekarang mari kita gunakan `saved_model_cli` untuk membuat prediksi untuk contoh yang baru saja kita simpan:

In [15]:
!saved_model_cli run --dir {model_path} --tag_set serve \
                     --signature_def serving_default    \
                     --inputs {input_name}=my_mnist_tests.npy

2021-02-18 22:15:30.294109: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-18 22:15:30.294306: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from my_mnist_model/0001/variables/variables
2021-02-18 22:15:30.323498: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
Result for output key dense_

In [16]:
np.round([[1.1347984e-04, 1.5187356e-07, 9.7032893e-04, 2.7640699e-03, 3.7826971e-06,
           7.6876910e-05, 3.9140293e-08, 9.9559116e-01, 5.3502394e-05, 4.2665208e-04],
          [8.2443521e-04, 3.5493889e-05, 9.8826385e-01, 7.0466995e-03, 1.2957400e-07,
           2.3389691e-04, 2.5639210e-03, 9.5886099e-10, 1.0314899e-03, 8.7952529e-08],
          [4.4693781e-05, 9.7028232e-01, 9.0526715e-03, 2.2641101e-03, 4.8766597e-04,
           2.8800720e-03, 2.2714981e-03, 8.3753867e-03, 4.0439744e-03, 2.9759688e-04]], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

## TensorFlow Serving

Instal [Docker] (https://docs.docker.com/install/) jika Anda belum memilikinya.Kemudian jalankan:

```Bash
Docker menarik tensorflow/serving
ekspor ml_path = $ home/ml # atau di mana pun proyek ini berada
Docker run -it --rm -p 8500: 8500 -p 8501: 8501 \
-v "$ ml_path/my_mnist_model:/model/my_mnist_model" \
-e model_name = my_mnist_model \
Tensorflow/serving
```


Setelah Anda selesai menggunakannya, tekan Ctrl-C untuk mematikan server.

Atau, jika `tensorflow_model_server` diinstal (mis., Jika Anda menjalankan buku catatan ini di Colab), maka 3 sel berikut akan memulai server:

In [17]:
os.environ["MODEL_DIR"] = os.path.split(os.path.abspath(model_path))[0]

In [18]:
%%bash --bg
nohup tensorflow_model_server \
     --rest_api_port=8501 \
     --model_name=my_mnist_model \
     --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [19]:
!tail server.log

2021-02-16 22:33:09.323538: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:93] Reading SavedModel debug info (if present) from: /models/my_mnist_model/0001
2021-02-16 22:33:09.323642: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-02-16 22:33:09.360572: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:206] Restoring SavedModel bundle.
2021-02-16 22:33:09.361764: I external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2200000000 Hz
2021-02-16 22:33:09.387713: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /models/my_mnist_model/0001
2021-02-16 22:33:09.

In [20]:
import json

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [21]:
repr(input_data_json)[:1500] + "..."

'\'{"signature_name": "serving_default", "instances": [[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0

Sekarang mari kita gunakan API REST TensorFlow Serving untuk membuat prediksi:

In [22]:
import requests

SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

In [23]:
response.keys()

dict_keys(['predictions'])

In [24]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

### Menggunakan API GRPC

In [25]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [26]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

In [27]:
response

outputs {
  key: "dense_1"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 3
      }
      dim {
        size: 10
      }
    }
    float_val: 0.00011425172124290839
    float_val: 1.513665068841874e-07
    float_val: 0.0009818424005061388
    float_val: 0.0027773496694862843
    float_val: 3.758880893656169e-06
    float_val: 7.6266449468676e-05
    float_val: 3.9139514740327286e-08
    float_val: 0.995561957359314
    float_val: 5.344580131350085e-05
    float_val: 0.00043088122038170695
    float_val: 0.0008194865076802671
    float_val: 3.5498320357874036e-05
    float_val: 0.9882420897483826
    float_val: 0.00705744931474328
    float_val: 1.2937064752804872e-07
    float_val: 0.00023402832448482513
    float_val: 0.0025743397418409586
    float_val: 9.668431610876382e-10
    float_val: 0.0010369382798671722
    float_val: 8.833576004008137e-08
    float_val: 4.441547571332194e-05
    float_val: 0.970328688621521
    float_val: 0.009044423699378967
    

Konversi respons ke tensor:

In [28]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

Atau ke array yang numpy jika klien Anda tidak menyertakan perpustakaan Tensorflow:

In [29]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
shape = [dim.size for dim in outputs_proto.tensor_shape.dim]
y_proba = np.array(outputs_proto.float_val).reshape(shape)
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

## Menyebarkan versi model baru

In [30]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 1s 748us/step - loss: 1.1567 - accuracy: 0.6691 - val_loss: 0.3418 - val_accuracy: 0.9042
Epoch 2/10
1719/1719 [==============================] - 1s 697us/step - loss: 0.3376 - accuracy: 0.9032 - val_loss: 0.2674 - val_accuracy: 0.9242
Epoch 3/10
1719/1719 [==============================] - 1s 676us/step - loss: 0.2779 - accuracy: 0.9187 - val_loss: 0.2227 - val_accuracy: 0.9368
Epoch 4/10
1719/1719 [==============================] - 1s 669us/step - loss: 0.2362 - accuracy: 0.9318 - val_loss: 0.2032 - val_accuracy: 0.9432
Epoch 5/10
1719/1719 [==============================] - 1s 670us/step - loss: 0.2109 - accuracy: 0.9389 - val_loss: 0.1833 - val_accuracy: 0.9482
Epoch 6/10
1719/1719 [==============================] - 1s 675us/step - loss: 0.1951 - accuracy: 0.9430 - val_loss: 0.1740 - val_accuracy: 0.9498
Epoch 7/10
1719/1719 [==============================] - 1s 667us/step - loss: 0.1799 - accuracy: 0.9474 - val_loss: 0.1605 -

In [31]:
model_version = "0002"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
model_path

'my_mnist_model/0002'

In [32]:
tf.saved_model.save(model, model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


In [33]:
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

my_mnist_model/
    0001/
        saved_model.pb
        variables/
            variables.data-00000-of-00001
            variables.index
        assets/
    0002/
        saved_model.pb
        variables/
            variables.data-00000-of-00001
            variables.index
        assets/


** PERINGATAN **: Anda mungkin perlu menunggu sebentar sebelum model baru dimuat oleh tensorflow porsi.

In [34]:
import requests

SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
            
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status()
response = response.json()

In [35]:
response.keys()

dict_keys(['predictions'])

In [36]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

# Menyebarkan model ke platform AI Google Cloud

Membuat Layanan Prediksi di GCP AI Platform
Google Cloud AI Platform dapat digunakan untuk menyajikan model TensorFlow. Langkah-langkah umumnya meliputi:
1.  Menyiapkan akun GCP dan proyek, termasuk penagihan.
2.  Membuat *bucket* Google Cloud Storage (GCS) untuk menyimpan SavedModels.
3.  Mengunggah folder SavedModel ke *bucket* GCS.
4.  Mengonfigurasi AI Platform dengan membuat model dan kemudian membuat versi model, menunjuk ke jalur SavedModel di GCS.
5.  AI Platform menangani penskalaan otomatis dan penyeimbangan beban.
6.  Untuk mengautentikasi aplikasi klien, **akun layanan** (*service account*) dengan hak akses terbatas (misalnya, peran ML Engine Developer) direkomendasikan daripada kredensial pengguna. Kunci privat akun layanan (file JSON) diunduh dan digunakan oleh aplikasi klien.
7.  Pustaka Klien Google API dapat digunakan untuk berinteraksi dengan layanan.

Ikuti instruksi dalam buku ini untuk menggunakan model ke platform Google Cloud AI, unduh kunci pribadi akun layanan dan simpan ke `my_service_account_private_key.json` di direktori proyek.Juga, perbarui `Project_id`:

In [37]:
project_id = "onyx-smoke-242003"

In [38]:
import googleapiclient.discovery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my_service_account_private_key.json"
model_id = "my_mnist_model"
model_path = "projects/{}/models/{}".format(project_id, model_id)
model_path += "/versions/v0001/" # if you want to run a specific version
ml_resource = googleapiclient.discovery.build("ml", "v1").projects()

In [39]:
def predict(X):
    input_data_json = {"signature_name": "serving_default",
                       "instances": X.tolist()}
    request = ml_resource.predict(name=model_path, body=input_data_json)
    response = request.execute()
    if "error" in response:
        raise RuntimeError(response["error"])
    return np.array([pred[output_name] for pred in response["predictions"]])

In [40]:
Y_probas = predict(X_new)
np.round(Y_probas, 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

# Menerapkan Model ke Perangkat Seluler atau Tertanam (TensorFlow Lite - TFLite) 
TFLite membantu menerapkan model ke perangkat seluler dan tertanam dengan tujuan:
* Mengurangi ukuran model (untuk mempersingkat waktu unduh dan mengurangi penggunaan RAM).
* Mengurangi jumlah komputasi (untuk mengurangi latensi, penggunaan baterai, dan pemanasan).
* Menyesuaikan model dengan batasan khusus perangka.
* **Konverter Model TFLite**: Mengambil SavedModel dan mengompresnya ke format berbasis FlatBuffers (`.tflite`). Ini mengoptimalkan model dengan memangkas operasi yang tidak diperlukan dan menggabungkan operasi jika memungkinkan.
* **Pengurangan Ukuran Model**:
    * Menggunakan arsitektur jaringan saraf yang lebih kecil.
    * Menggunakan lebar bit yang lebih kecil (misalnya, *half-float* 16-bit).
    * **Kuantisasi**: Mengubah bobot menjadi bilangan bulat titik tetap 8-bi.
        * ***Post-training quantization***: Mengkuantisasi bobot setelah pelatihan menggunakan teknik kuantisasi simetris. Mengurangi ukuran model secara signifikan, tetapi bobot yang dikuantisasi dikonversi kembali ke *float* saat *runtime* (kecuali jika aktivasi juga dikuantisasi).
        * **Kuantisasi penuh**: Mengkuantisasi bobot dan aktivasi memungkinkan komputasi dilakukan sepenuhnya dengan bilangan bulat, menghasilkan percepatan yang signifikan. Memerlukan langkah kalibrasi menggunakan sampel data pelatihan yang representatif.
        * ***Quantization-aware training***: Menambahkan operasi kuantisasi palsu ke model selama pelatihan sehingga model dapat belajar mengabaikan *noise* kuantisasi, menghasilkan bobot akhir yang lebih tahan terhadap kuantisasi.


# TensorFlow di Browser (TensorFlow.js) 
Memungkinkan model berjalan langsung di browser pengguna. Berguna untuk konektivitas yang terputus-putus/lambat, respons latensi rendah, atau privasi pengguna.
* Alat `tensorflowjs_converter` mengonversi SavedModel atau file model Keras ke format TensorFlow.js Layers (direktori berisi file bobot *sharded* dan file `model.json`).

# Menggunakan GPU

Pelatihan jaringan saraf yang besar bisa sangat lambat. GPU dapat mempercepat pelatihan secara signifikan.
* **Mendapatkan GPU**: Beli sendiri (kartu Nvidia dengan CUDA Compute Capability 3.5+) atau gunakan VM yang dilengkapi GPU di *cloud* (misalnya, Colaboratory).
* Perlu menginstal driver Nvidia, CUDA, dan cuDNN (pustaka primitif yang dipercepat GPU untuk DNN).
* **Mengelola RAM GPU**:
    * Secara default, TensorFlow mengambil semua RAM di semua GPU yang tersedia.
    * Variabel lingkungan `CUDA_VISIBLE_DEVICES` dapat digunakan untuk menetapkan GPU tertentu ke proses tertentu.
    * `tf.config.experimental.set_virtual_device_configuration()` dengan `VirtualDeviceConfiguration(memory_limit=...)` untuk membatasi RAM yang diambil TensorFlow pada GPU.
    * `tf.config.experimental.set_memory_growth(gpu, True)` agar TensorFlow hanya mengambil memori saat dibutuhkan.
* **Menempatkan Operasi dan Variabel pada Perangkat**:
    * TensorFlow memiliki algoritma penempatan dinamis (meskipun sekarang kurang digunakan), tetapi `tf.keras` dan `tf.data` umumnya melakukan penempatan yang baik (pra-pemrosesan data di CPU, operasi jaringan saraf di GPU).
    * Penempatan manual dapat dilakukan menggunakan konteks `tf.device("/gpu:0")` atau `tf.device("/cpu:0")`.
    * `tf.config.set_soft_device_placement(True)` memungkinkan *fallback* ke CPU jika penempatan GPU yang diminta gagal.
* **Eksekusi Paralel Lintas Beberapa Perangkat**: TensorFlow menganalisis grafik TF Function untuk menemukan operasi yang dapat dijalankan secara paralel pada perangkat yang berbeda menggunakan antrean evaluasi dan *thread pool* (inter-op dan intra-op).


** Catatan **: `tf.test.is_gpu_available ()` sudah usang.Sebagai gantinya, silakan gunakan `tf.config.list_physical_devices ('gpu')`.

In [41]:
# tf.test.is_gpu_available () # usang
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [42]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [43]:
tf.test.is_built_with_cuda()

True

In [44]:
from tensorflow.python.client.device_lib import list_local_devices

devices = list_local_devices()
devices

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7325002731160755624,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11139884032
 locality {
   bus_id: 1
   links {
     link {
       device_id: 1
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 7150956550266107441
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7",
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 11139884032
 locality {
   bus_id: 1
   links {
     link {
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 15909479382059415698
 physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:00:05.0, compute capability: 3.7"]

# Pelatihan Terdistribusi

* **Paralelisme Model**: Model dipecah menjadi beberapa bagian dan setiap bagian dijalankan pada perangkat yang berbeda. Sulit dan sangat bergantung pada arsitektur jaringan. Umumnya tidak banyak manfaat untuk jaringan yang terhubung penuh karena komunikasi lintas perangkat yang tinggi. Mungkin lebih baik untuk CNN atau RNN tertentu.
* **Paralelisme Data**: Model direplikasi di setiap perangkat, dan setiap replika dilatih pada subset data yang berbeda. Gradien yang dihitung oleh setiap replika kemudian dirata-ratakan, dan hasilnya digunakan untuk memperbarui parameter model.
    * **Strategi Tercermin (*Mirrored Strategy*)**: `tf.distribute.MirroredStrategy()`. Parameter model sepenuhnya dicerminkan di semua GPU, dan pembaruan parameter yang sama persis diterapkan pada setiap GPU (pembaruan sinkron). Menggunakan algoritma **AllReduce** (misalnya, NCCL) untuk menghitung rata-rata gradien secara efisien.
    * **Parameter Terpusat**: `tf.distribute.experimental.CentralStorageStrategy()`. Parameter model disimpan di luar perangkat pekerja (misalnya, di CPU atau server parameter).
        * ***Pembaruan Sinkron***: Agregator menunggu semua gradien sebelum menghitung rata-rata dan memperbarui parameter. Dapat diperlambat oleh replika yang paling lamba.
        * ***Pembaruan Asinkron***: Setiap replika memperbarui parameter model secara independen segera setelah selesai menghitung gradien. Lebih banyak langkah pelatihan per menit tetapi dapat menderita **gradien basi** (*stale gradients*), yang dapat memperlambat konvergensi atau menyebabkan divergensi.
    * **Saturasi *Bandwidth***: Menjadi masalah ketika mentransfer parameter dan gradien, terutama untuk model padat yang besar. Menggunakan presisi *float* 16-bit (`tf.bfloat16`) dapat membantu.
* **API Strategi Distribusi**:
    * `tf.distribute.MirroredStrategy()`: Untuk beberapa GPU di satu mesin (atau beberapa mesin dengan `MultiWorkerMirroredStrategy`).
    * `tf.distribute.experimental.CentralStorageStrategy()`: Untuk parameter terpusat di satu mesin.
    * `tf.distribute.experimental.ParameterServerStrategy()`: Untuk paralelisme data asinkron dengan server parameter di banyak mesin.
    * `tf.distribute.experimental.TPUStrategy()`: Untuk TPU.
* **Melatih Model di Klaster TensorFlow**:
    * Klaster TensorFlow adalah sekelompok proses TensorFlow yang berjalan secara paralel, biasanya di mesin yang berbeda. Setiap proses TF disebut **tugas** (*task*) atau **server TF**, dengan alamat IP, port, dan **tipe** (misalnya, `"worker"`, `"chief"`, `"ps"`, `"evaluator"`).
    * Spesifikasi klaster didefinisikan (misalnya, melalui variabel lingkungan `TF_CONFIG` dalam format JSON) untuk setiap tugas.
* **Menjalankan Pekerjaan Pelatihan Besar di Google Cloud AI Platform**:
    * AI Platform menangani penyediaan dan konfigurasi VM GPU.
    * Gunakan alat baris perintah `gcloud` untuk mengirimkan pekerjaan pelatihan.
    * **Penyetelan *Hyperparameter Black Box* di AI Platform (Google Vizier)**:
        * Menggunakan file konfigurasi YAML untuk mendefinisikan ruang pencarian *hyperparameter*, tujuan (misalnya, `MAXIMIZE`), `hyperparameterMetricTag`, dll.
        * AI Platform memantau direktori output untuk *event file* yang berisi metrik yang ditentukan dan menggunakan optimasi Bayesian untuk menyarankan nilai *hyperparameter* untuk uji coba berikutnya.

In [45]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [46]:
def create_model():
    return keras.models.Sequential([
        keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                            padding="same", input_shape=[28, 28, 1]),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"), 
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dense(units=64, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])

In [47]:
batch_size = 100
model = create_model()
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

Epoch 1/10
550/550 [==============================] - 11s 18ms/step - loss: 1.8163 - accuracy: 0.3979 - val_loss: 0.3446 - val_accuracy: 0.9010
Epoch 2/10
550/550 [==============================] - 9s 17ms/step - loss: 0.4949 - accuracy: 0.8482 - val_loss: 0.1962 - val_accuracy: 0.9458
Epoch 3/10
550/550 [==============================] - 10s 17ms/step - loss: 0.3345 - accuracy: 0.9012 - val_loss: 0.1343 - val_accuracy: 0.9622
Epoch 4/10
550/550 [==============================] - 10s 17ms/step - loss: 0.2537 - accuracy: 0.9267 - val_loss: 0.1049 - val_accuracy: 0.9718
Epoch 5/10
550/550 [==============================] - 10s 17ms/step - loss: 0.2099 - accuracy: 0.9394 - val_loss: 0.0875 - val_accuracy: 0.9752
Epoch 6/10
550/550 [==============================] - 10s 17ms/step - loss: 0.1901 - accuracy: 0.9439 - val_loss: 0.0797 - val_accuracy: 0.9772
Epoch 7/10
550/550 [==============================] - 10s 18ms/step - loss: 0.1672 - accuracy: 0.9506 - val_loss: 0.0745 - val_accuracy: 

In [48]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

distribution = tf.distribute.MirroredStrategy()

# Ubah algoritma all-reduce default:
# distribusi = tf.distribute.mirroredstrategy (
# cross_device_ops = tf.distribute.hierarchicalCopyAlluce ())

# Tentukan daftar GPU untuk digunakan:
# distribusi = tf.distribute.mirroredstrategy (perangkat = ["/gpu: 0", "/gpu: 1"])

# Gunakan strategi penyimpanan pusat sebagai gantinya:
# distribusi = tf.distribute.experimental.centralstoragestry ()

# if is_colab dan "colab_tpu_addr" di os.environ:
# tpu_address = "grpc: //" + os.environ ["colab_tpu_addr"]
# kalau tidak:
# tpu_address = ""
# resolver = tf.distribute.cluster_resolver.tpuclusterResolver (tpu_address)
# tf.config.experimental_connect_to_cluster (resolver)
# tf.tpu.experimental.initialize_tpu_system (resolver)
# distribusi = tf.distribute.experimental.tpustrategy (resolver)

with distribution.scope():
    model = create_model()
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [49]:
batch_size = 100 # must be divisible by the number of workers
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

Epoch 1/10
INFO:tensorflow:batch_all_reduce: 10 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 10 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/t

In [50]:
model.predict(X_new)

array([[2.53707055e-10, 7.94509292e-10, 1.02021443e-06, 3.37102080e-08,
        4.90816797e-11, 4.37713789e-11, 2.43314297e-14, 9.99996424e-01,
        1.50591750e-09, 2.50736753e-06],
       [1.11715025e-07, 8.56921833e-05, 9.99914169e-01, 6.31697228e-09,
        3.99949344e-11, 4.47976906e-10, 8.46022008e-09, 3.03771834e-08,
        2.91782563e-08, 1.95555502e-10],
       [4.68117065e-07, 9.99787748e-01, 1.01387537e-04, 2.87393277e-06,
        5.29725839e-05, 1.55926125e-06, 2.07211669e-05, 1.76809226e-05,
        9.37155255e-06, 5.19965897e-06]], dtype=float32)

Loop Pelatihan Kustom:

In [51]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

K = keras.backend

distribution = tf.distribute.MirroredStrategy()

with distribution.scope():
    model = create_model()
    optimizer = keras.optimizers.SGD()

with distribution.scope():
    dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).repeat().batch(batch_size)
    input_iterator = distribution.make_dataset_iterator(dataset)
    
@tf.function
def train_step():
    def step_fn(inputs):
        X, y = inputs
        with tf.GradientTape() as tape:
            Y_proba = model(X)
            loss = K.sum(keras.losses.sparse_categorical_crossentropy(y, Y_proba)) / batch_size

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        return loss

    per_replica_losses = distribution.experimental_run(step_fn, input_iterator)
    mean_loss = distribution.reduce(tf.distribute.ReduceOp.SUM,
                                    per_replica_losses, axis=None)
    return mean_loss

n_epochs = 10
with distribution.scope():
    input_iterator.initialize()
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))
        for iteration in range(len(X_train) // batch_size):
            print("\rLoss: {:.3f}".format(train_step().numpy()), end="")
        print()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Instructions for updating:
Use the iterator's `initializer` property instead.
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 10 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 10 all-reduces with algorithm = nccl, num_packs = 1
Loss: 0.380
Epoch 2/10
Loss: 0.302
Epoch 3/10
Loss: 0.285
Epoch 4/10
Loss: 0.294
Epoch 5/10
Loss: 0.304
Epoch 6/10
Loss: 0.310
Epoch 7/10
Loss: 0.310
Epoch 8/10
Loss: 0.306
Epoch 9/10
Loss: 0.303
Epoch 10/10
Loss: 0.298


## Pelatihan di beberapa server

Cluster TensorFlow adalah sekelompok proses TensorFlow yang berjalan secara paralel, biasanya pada mesin yang berbeda, dan berbicara satu sama lain untuk menyelesaikan beberapa pekerjaan, misalnya melatih atau melaksanakan jaringan saraf.Setiap proses TF dalam cluster disebut "tugas" (atau "server TF").Ini memiliki alamat IP, port, dan jenis (juga disebut perannya atau tugasnya).Jenisnya bisa `" pekerja "`, `" chief "`, `" ps "` (server parameter) atau `" evaluator "`:
*Masing -masing ** Pekerja ** melakukan perhitungan, biasanya pada mesin dengan satu atau lebih GPU.
*The ** Chief ** melakukan perhitungan juga, tetapi juga menangani pekerjaan tambahan seperti menulis log tensorboard atau menyimpan pos pemeriksaan.Ada satu kepala dalam sebuah cluster, biasanya pekerja pertama (mis., Pekerja #0).
*A ** Server Parameter ** (PS) hanya melacak nilai variabel, biasanya pada mesin hanya CPU.
*Evaluator ** ** jelas menangani evaluasi.Biasanya ada evaluator tunggal dalam sebuah cluster.

Set tugas yang berbagi jenis yang sama sering disebut "pekerjaan".Misalnya, pekerjaan "pekerja" adalah himpunan semua pekerja.

Untuk memulai kluster TensorFlow, Anda harus terlebih dahulu mendefinisikannya.Ini berarti menentukan semua tugas (alamat IP, port TCP, dan jenis).Misalnya, spesifikasi cluster berikut mendefinisikan cluster dengan 3 tugas (2 pekerja dan 1 server parameter).Ini adalah kamus dengan satu kunci per pekerjaan, dan nilainya adalah daftar alamat tugas:

In [52]:
cluster_spec = {
    "worker": [
        "machine-a.example.com:2222",  # /job:worker/task:0
        "machine-b.example.com:2222"   # /job:worker/task:1
    ],
    "ps": ["machine-c.example.com:2222"] # /job:ps/task:0
}

Setiap tugas dalam cluster dapat berkomunikasi dengan setiap tugas lain di server, jadi pastikan untuk mengkonfigurasi firewall Anda untuk mengesahkan semua komunikasi antara mesin -mesin ini di port ini (biasanya lebih sederhana jika Anda menggunakan port yang sama di setiap mesin).

Ketika suatu tugas dimulai, perlu diberitahu yang mana: tipe dan indeksnya (indeks tugas juga disebut ID tugas).Cara umum untuk menentukan semuanya sekaligus (baik spesifikasi cluster dan tipe dan ID tugas saat ini) adalah dengan mengatur variabel lingkungan `tf_config` sebelum memulai program.Ini harus menjadi kamus yang dikodekan JSON yang berisi spesifikasi cluster (di bawah `" cluster "` kunci), dan jenis dan indeks tugas untuk memulai (di bawah tugas `" Tugas "`).Misalnya, variabel lingkungan `tf_config` berikut mendefinisikan cluster yang sama seperti di atas, dengan 2 pekerja dan 1 server parameter, dan menentukan bahwa tugas untuk memulai adalah pekerja #1:

In [53]:
import os
import json

os.environ["TF_CONFIG"] = json.dumps({
    "cluster": cluster_spec,
    "task": {"type": "worker", "index": 1}
})
os.environ["TF_CONFIG"]

'{"cluster": {"worker": ["machine-a.example.com:2222", "machine-b.example.com:2222"], "ps": ["machine-c.example.com:2222"]}, "task": {"type": "worker", "index": 1}}'

Beberapa platform (mis., Google Cloud ML Engine) secara otomatis mengatur variabel lingkungan ini untuk Anda.

Kelas TensorFlow `TFConfigClusterResolver` membaca konfigurasi cluster dari variabel lingkungan ini:

In [54]:
import tensorflow as tf

resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
resolver.cluster_spec()

ClusterSpec({'ps': ['machine-c.example.com:2222'], 'worker': ['machine-a.example.com:2222', 'machine-b.example.com:2222']})

In [55]:
resolver.task_type

'worker'

In [56]:
resolver.task_id

1

Sekarang mari kita jalankan cluster yang lebih sederhana dengan hanya dua tugas pekerja, keduanya berjalan di mesin lokal. kita akan menggunakan `multiworkermirroredstrategy` untuk melatih model di kedua tugas ini.

Langkah pertama adalah menulis kode pelatihan.Karena kode ini akan digunakan untuk menjalankan kedua pekerja, masing -masing dalam prosesnya sendiri, kita menulis kode ini ke file python terpisah, `my_mnist_multiworker_task.py`.Kode ini relatif mudah, tetapi ada beberapa hal penting yang perlu diperhatikan:
* kita membuat `multiworkerMirroredStrategy` sebelum melakukan hal lain dengan TensorFlow.
* Hanya satu pekerja yang akan mengurus logging ke Tensorboard dan menyimpan pos pemeriksaan.Seperti disebutkan sebelumnya, pekerja ini disebut *kepala *, dan dengan konvensi biasanya pekerja #0.

In [57]:
%%writefile my_mnist_multiworker_task.py

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time

# Di awal program
distribution = tf.distribute.MultiWorkerMirroredStrategy()

resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
print("Starting task {}{}".format(resolver.task_type, resolver.task_id))

# Hanya pekerja #0 yang akan menulis pos pemeriksaan dan log ke tensorboard
if resolver.task_id == 0:
    root_logdir = os.path.join(os.curdir, "my_mnist_multiworker_logs")
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    run_dir = os.path.join(root_logdir, run_id)
    callbacks = [
        keras.callbacks.TensorBoard(run_dir),
        keras.callbacks.ModelCheckpoint("my_mnist_multiworker_model.h5",
                                        save_best_only=True),
    ]
else:
    callbacks = []

# Muat dan persiapkan dataset MNIST
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis] / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

with distribution.scope():
    model = keras.models.Sequential([
        keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                            padding="same", input_shape=[28, 28, 1]),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"), 
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dense(units=64, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
          epochs=10, callbacks=callbacks)

Overwriting my_mnist_multiworker_task.py


Dalam aplikasi dunia nyata, biasanya akan ada satu pekerja per mesin, tetapi dalam contoh ini kita menjalankan kedua pekerja di mesin yang sama, sehingga mereka berdua akan mencoba menggunakan semua RAM GPU yang tersedia (jika mesin ini memiliki GPU), dan ini kemungkinan akan menyebabkan kesalahan out-of-memory (OOM).Untuk menghindari ini, kita dapat menggunakan variabel lingkungan `cuda_visible_devices` untuk menetapkan GPU yang berbeda untuk setiap pekerja.Atau, kita dapat menonaktifkan dukungan GPU, seperti ini:

In [58]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

kita sekarang siap untuk memulai kedua pekerja, masing -masing dalam prosesnya sendiri, menggunakan modul `Subprocess` Python.Sebelum kita memulai setiap proses, kita perlu mengatur variabel lingkungan `tf_config` dengan tepat, hanya mengubah indeks tugas:

In [59]:
import subprocess

cluster_spec = {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]}

for index, worker_address in enumerate(cluster_spec["worker"]):
    os.environ["TF_CONFIG"] = json.dumps({
        "cluster": cluster_spec,
        "task": {"type": "worker", "index": index}
    })
    subprocess.Popen("python my_mnist_multiworker_task.py", shell=True)

Itu saja!Cluster TensorFlow kita sekarang berjalan, tetapi kita tidak dapat melihatnya di buku catatan ini karena berjalan dalam proses terpisah (tetapi jika Anda menjalankan buku catatan ini di Jupyter, Anda dapat melihat log pekerja di log server Jupyter).

Karena Kepala (Pekerja #0) menulis ke Tensorboard, kita menggunakan Tensorboard untuk melihat kemajuan pelatihan.Jalankan sel berikut, lalu klik tombol Pengaturan (mis., Ikon Gear) di antarmuka Tensorboard dan centang kotak "Data Muat Ulang" untuk membuat papan tensor secara otomatis menyegarkan setiap 30 -an.Setelah zaman pertama pelatihan selesai (yang mungkin memakan waktu beberapa menit), dan setelah Tensorboard menyegarkan, tab Scalars akan muncul.Klik pada tab ini untuk melihat kemajuan pelatihan model dan akurasi validasi.

In [60]:
%load_ext tensorboard
%tensorboard --logdir=./my_mnist_multiworker_logs --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Itu saja!Setelah pelatihan selesai, pos pemeriksaan terbaik dari model akan tersedia di file `my_mnist_multiworker_model.h5`.Anda dapat memuatnya menggunakan `keras.models.load_model ()` dan menggunakannya untuk prediksi, seperti biasa:

In [61]:
from tensorflow import keras

model = keras.models.load_model("my_mnist_multiworker_model.h5")
Y_pred = model.predict(X_new)
np.argmax(Y_pred, axis=-1)

array([7, 2, 1])

Dan itu saja untuk hari ini!Semoga Anda menemukan ini bermanfaat.😊

# Solusi Latihan

1.  **Isi SavedModel dan Cara Memeriksanya**: SavedModel berisi model TensorFlow, termasuk arsitekturnya (grafik komputasi) dan bobotnya.  Disimpan sebagai direktori yang berisi file `saved_model.pb` dan subdirektori `variables`.  Mungkin juga menyertakan subdirektori `assets`.  SavedModel dapat berisi satu atau lebih *metagraph* (grafik komputasi ditambah definisi tanda tangan fungsi).  Untuk memeriksanya, gunakan alat baris perintah `saved_model_cli` atau muat menggunakan `tf.saved_model.load()` dan periksa di Python. 
2.  **Kapan Menggunakan TF Serving dan Fitur Utamanya**: TF Serving memungkinkan Anda menerapkan beberapa model TensorFlow (atau beberapa versi dari model yang sama) dan membuatnya mudah diakses oleh semua aplikasi Anda melalui REST API atau gRPC API.  Fitur utamanya termasuk pemantauan direktori dan penerapan otomatis model baru, peralihan versi model yang mulus, dukungan untuk pengujian A/B dan penerapan terbatas (*canary*), kecepatan, pengujian yang baik, skalabilitas yang sangat baik, dan kemampuan untuk mengelompokkan permintaan individu menjadi batch.  Untuk menerapkannya, Anda dapat menggunakan gambar Docker atau solusi yang dihosting sepenuhnya seperti Google Cloud AI Platform. 
3.  **Menerapkan Model di Beberapa Instans TF Serving**: Yang perlu Anda lakukan hanyalah mengonfigurasi instans TF Serving ini untuk memantau direktori model yang sama, lalu mengekspor model baru Anda sebagai SavedModel ke subdirektori. 
4.  **Kapan Menggunakan gRPC API vs. REST API dengan TF Serving**: gRPC API lebih efisien daripada REST API.  Namun, pustaka kliennya tidak tersedia secara luas, dan jika Anda mengaktifkan kompresi saat menggunakan REST API, Anda bisa mendapatkan kinerja yang hampir sama.  Jadi, gRPC API paling berguna ketika Anda membutuhkan kinerja setinggi mungkin dan klien tidak terbatas pada REST API. 
5.  **Cara TFLite Mengurangi Ukuran Model**:
    * Konverter mengoptimalkan SavedModel, menyusutkannya dan mengurangi latensinya dengan memangkas operasi yang tidak perlu dan menggabungkan operasi. 
    * Kuantisasi pasca-pelatihan secara dramatis mengurangi ukuran model. 
    * Menyimpan model yang dioptimalkan menggunakan format FlatBuffer, yang dapat dimuat langsung ke RAM tanpa penguraian, mengurangi waktu muat dan jejak memori. 
6.  **Pelatihan Sadar Kuantisasi (*Quantization-Aware Training*)**: Melibatkan penambahan operasi kuantisasi palsu ke model selama pelatihan.  Ini memungkinkan model untuk belajar mengabaikan *noise* kuantisasi; bobot akhir akan lebih kuat terhadap kuantisasi. 
7.  **Paralelisme Model vs. Paralelisme Data**:
    * Paralelisme Model: Memecah model Anda menjadi beberapa bagian dan menjalankannya secara paralel di beberapa perangkat. 
    * Paralelisme Data: Membuat beberapa replika persis dari model Anda dan menerapkannya di beberapa perangkat.  Pada setiap iterasi selama pelatihan, setiap replika diberi batch data yang berbeda, dan ia menghitung gradien kerugian terhadap parameter model. 
    * Paralelisme data umumnya direkomendasikan karena memerlukan lebih sedikit komunikasi antar perangkat, lebih mudah diimplementasikan, dan berfungsi dengan cara yang sama untuk model apa pun. 
8.  **Strategi Distribusi untuk Pelatihan di Beberapa Server**:
    * **MultiWorkerMirroredStrategy**: Melakukan paralelisme data cermin.  Model direplikasi di semua server dan perangkat yang tersedia, dan setiap replika mendapatkan batch data yang berbeda dan menghitung gradiennya sendiri.  Rata-rata gradien dihitung dan dibagikan menggunakan implementasi AllReduce terdistribusi (NCCL secara default), dan semua replika melakukan langkah *Gradient Descent* yang sama.  Ini adalah yang paling sederhana untuk digunakan dan berkinerja cukup baik. 
    * **ParameterServerStrategy**: Melakukan paralelisme data asinkron.  Model direplikasi di semua perangkat di semua pekerja, dan parameter dipecah di semua server parameter.  Setiap pekerja menjalankan loop pelatihannya sendiri secara asinkron.  Umumnya lebih lambat dari strategi sebelumnya. 
    * Anda harus menggunakan **MultiWorkerMirroredStrategy** secara umum.  Batasan utamanya adalah ia mengharuskan model muat dalam RAM pada setiap replika.  **ParameterServerStrategy** berguna untuk melatih model besar yang tidak muat dalam RAM GPU. 

## 9.
_Exercise: Latih model (model apa pun yang Anda suka) dan sebarkan ke platform TF atau Google Cloud AI.Tulis kode klien untuk menanyakannya menggunakan API REST atau API GRPC.Perbarui model dan gunakan versi baru.Kode klien Anda sekarang akan meminta versi baru.Kembali ke versi pertama._

Harap ikuti langkah-langkah di <a href = "#deploying-tensorflow-model-to-tensorflow-serving- (tfs)"> Menyebarkan model tensorflow ke bagian tensorflow yang melayani </a> di atas.

# 10.
_Exercise: Latih model apa pun di beberapa GPU pada mesin yang sama menggunakan `mirroredstrategy` (jika Anda tidak memiliki akses ke GPU, Anda dapat menggunakan colaboratory dengan runtime GPU dan membuat dua GPU virtual).Latih model lagi menggunakan `CentralStoragestry` dan bandingkan waktu pelatihan._

Harap ikuti langkah-langkah di bagian [Pelatihan Terdistribusi] (#-Training Terdistribusi) di atas.

# 11.
_Exercise: Latih model kecil di platform Google Cloud AI, menggunakan Black Box Hyperparameter Tuning._

Silakan ikuti instruksi di halaman 716-717 dari buku ini.Anda juga dapat membaca [halaman dokumentasi ini] (https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-oveview) dan melalui contoh di [blog yang bagus ini iniPOST] (https://towardsdatacience.com/how-to-do-bayesian-hyper-parameter-tuning-on-a-blackbox-model-882009552c6d) oleh Lak Lakshmanan.